In [1]:
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time
import error_table

from sampling_utils import *
from gen_utils import *
from evaluation_utils import *
from config_db import ConfigDB

import ghalton
import random

In [2]:
num_pics = 500
min_num_cars = 1
max_num_cars = 4

domains = [[52,87],[1,37],[0,1],[0.35,1],[0.5,1.5],[0.25,1],[0.5,1],[0.5,1]]
types = ['int','int','float','float','float','float','float','float']

gen_misclass = True
store_config_db = False
store_err_tab = True

IOMIN_THRESH = 0.4
IOU_THRESH = 0.5
AP_THRESH = 0.75
REC_THRESH = 0.75
NN_PROB_THRESH = 0.5

if gen_misclass:
    checkpoint = '/home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000'
    import squeezedet as nn
    net = nn.init(checkpoint)

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
INFO:tensorflow:Restoring parameters from /home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000


In [20]:
for j in range(2,max_num_cars):
    
    total_attempts = 0
    rejections = 0
    
    # Init data structure
    if store_err_tab:
        err_table = error_table()
    if store_config_db:
        config_db = ConfigDB()
    
    path_data_set = './data/mis_halt/'
    #path_data_set = '/home/tommaso/analyzeNN/data/train_0/mis/mis_' + str(j) + '/'
    
    i = 0
    v = 0
    
    sequence_len = 1000000
    sequencer = ghalton.Halton(2*j+4)
    points = sequencer.get(sequence_len)

    while i < num_pics and v < sequence_len:
        
        v = v+1

        # Sample configuration
        #num_cars = random.randint(min_num_cars, max_num_cars)
        num_cars = j
        #sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
        #sample, norm_sample = get_sample(v, 'halton', types=types, domains=domains, dims=sample_dims)\
        sample = []
        # Background
        sample += [[random.randint(domains[0][0], domains[0][1]-1)]]
        # Cars
        cars = []
        for z in range(0,num_cars):
            cars += [random.randint(domains[1][0], domains[1][1]-1)]
        sample += [cars]
        
        point = points[v]
        
        adapt_point = []            
        
        for z in range(num_cars):
            adapt_point += [point[z]*(domains[2][1]-domains[2][0])+domains[2][0]]
        for z in range(num_cars):
            adapt_point += [point[num_cars+z]*(domains[3][1]-domains[3][0])+domains[3][0]]
        for z in range(4):
            adapt_point += [point[2*num_cars+z]*(domains[4+z][1]-domains[4+z][0])+domains[4+z][0]]
            
        sample += [adapt_point[0:num_cars]]   
        sample += [adapt_point[num_cars:2*num_cars]]
        sample += [[adapt_point[2*num_cars]]]
        sample += [[adapt_point[2*num_cars+1]]]
        sample += [[adapt_point[2*num_cars+2]]]
        sample += [[adapt_point[2*num_cars+3]]]            
        
        sample[3].sort(reverse=True)
            
            
        # Generate image from sample
        img, ground_boxes, img_info = gen_image(sample)

        # Is it a good picture? (cars don't overlap too much)
        save_img = True
        for m in range(len(ground_boxes)):
            for n in range(m+1, len(ground_boxes)):
                if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                    save_img = False

        ap = 0
        recall = 0

        # Check misclassification
        if gen_misclass and save_img:
            # Save tmp image    
            tmp_img_file_name = './tmp.png'
            img.save(tmp_img_file_name)

            # Classify img
            (boxes,probs,cats) = nn.classify(tmp_img_file_name, net, NN_PROB_THRESH) 

            # Not cars
            if any(cats):
                save_img = True
            else:
                ap, recall = average_precision(ground_boxes, boxes, IOU_THRESH)
                save_img =  (ap <= AP_THRESH or recall <= REC_THRESH)
                
                        

        if save_img:
            
            print str(ap),str(recall)

            if store_config_db:
                config_db.add(sample_pad)            

            file_name = 'm_h_' + str(j) + '_' + str(i).zfill(6)
            #file_name = 'm_d_' + str(i).zfill(6)

            print(file_name + ' ' + str(ap) + ' ' + str(recall))

            # Save data
            save_image(img, file_name, path_data_set)
            save_label(ground_boxes, file_name, path_data_set)

            # Save in image set
            with open(path_data_set + 'image_set_m_1.txt', "a+") as im_set:
                im_set.write(file_name + '\n')
                im_set.close()

            if store_err_tab:
                err_table.update_with_elem(img_info)

            i+=1
    else:
        rejections += 1

#         if(total_attempts %50 == 0):
#             print('gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
print('END gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))

if store_err_tab:
    err_table_file = open(path_data_set + 'error_table' + str(),'w')
    pickle.dump(err_table,err_table_file)
    err_table_file.close()

0.666666666667 1.0
m_h_2_000000 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000001 0.666666666667 1.0
0.5 0.5
m_h_2_000002 0.5 0.5
0.666666666667 1.0
m_h_2_000003 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000004 0.666666666667 1.0
0.5 0.5
m_h_2_000005 0.5 0.5
0.5 0.5
m_h_2_000006 0.5 0.5
0.666666666667 1.0
m_h_2_000007 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000008 0.666666666667 1.0
0.5 0.5
m_h_2_000009 0.5 0.5
0.666666666667 1.0
m_h_2_000010 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000011 0.666666666667 1.0
0.0 0.0
m_h_2_000012 0.0 0.0
0.666666666667 1.0
m_h_2_000013 0.666666666667 1.0
0.5 1.0
m_h_2_000014 0.5 1.0
0.5 1.0
m_h_2_000015 0.5 1.0
0.666666666667 1.0
m_h_2_000016 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000017 0.666666666667 1.0
0.5 1.0
m_h_2_000018 0.5 1.0
0.666666666667 1.0
m_h_2_000019 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000020 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000021 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000022 0.666666666667 1.0
0.5

0.5 0.5
m_h_2_000189 0.5 0.5
0.5 0.5
m_h_2_000190 0.5 0.5
0.666666666667 1.0
m_h_2_000191 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000192 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000193 0.666666666667 1.0
0.5 0.5
m_h_2_000194 0.5 0.5
0.666666666667 1.0
m_h_2_000195 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000196 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000197 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000198 0.666666666667 1.0
0.333333333333 0.5
m_h_2_000199 0.333333333333 0.5
0.666666666667 1.0
m_h_2_000200 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000201 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000202 0.666666666667 1.0
0.5 0.5
m_h_2_000203 0.5 0.5
0.666666666667 1.0
m_h_2_000204 0.666666666667 1.0
0.5 0.5
m_h_2_000205 0.5 0.5
0.5 1.0
m_h_2_000206 0.5 1.0
1.0 0.5
m_h_2_000207 1.0 0.5
0.666666666667 1.0
m_h_2_000208 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000209 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000210 0.666666666667 1.0
0.5 0.5
m_h_2_000211 0.5 0.5
0.3

0.666666666667 1.0
m_h_2_000372 0.666666666667 1.0
0.0 0.0
m_h_2_000373 0.0 0.0
0.666666666667 1.0
m_h_2_000374 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000375 0.666666666667 1.0
0.5 0.5
m_h_2_000376 0.5 0.5
0.666666666667 1.0
m_h_2_000377 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000378 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000379 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000380 0.666666666667 1.0
0.333333333333 0.5
m_h_2_000381 0.333333333333 0.5
0.666666666667 1.0
m_h_2_000382 0.666666666667 1.0
0.5 0.5
m_h_2_000383 0.5 0.5
0.666666666667 1.0
m_h_2_000384 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000385 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000386 0.666666666667 1.0
0.5 0.5
m_h_2_000387 0.5 0.5
0.666666666667 1.0
m_h_2_000388 0.666666666667 1.0
0.666666666667 1.0
m_h_2_000389 0.666666666667 1.0
0.5 0.5
m_h_2_000390 0.5 0.5
0.5 1.0
m_h_2_000391 0.5 1.0
0.666666666667 1.0
m_h_2_000392 0.666666666667 1.0
1.0 0.5
m_h_2_000393 1.0 0.5
0.666666666667 1.0
m_h_2_000394 

0.75 1.0
m_h_3_000063 0.75 1.0
0.5 0.666666666667
m_h_3_000064 0.5 0.666666666667
0.666666666667 0.666666666667
m_h_3_000065 0.666666666667 0.666666666667
1.0 0.666666666667
m_h_3_000066 1.0 0.666666666667
0.666666666667 0.666666666667
m_h_3_000067 0.666666666667 0.666666666667
0.5 0.666666666667
m_h_3_000068 0.5 0.666666666667
0.2 0.333333333333
m_h_3_000069 0.2 0.333333333333
0.5 0.333333333333
m_h_3_000070 0.5 0.333333333333
0.666666666667 0.666666666667
m_h_3_000071 0.666666666667 0.666666666667
0.666666666667 0.666666666667
m_h_3_000072 0.666666666667 0.666666666667
0.75 1.0
m_h_3_000073 0.75 1.0
0.75 1.0
m_h_3_000074 0.75 1.0
0.5 0.666666666667
m_h_3_000075 0.5 0.666666666667
0.5 0.333333333333
m_h_3_000076 0.5 0.333333333333
0.75 1.0
m_h_3_000077 0.75 1.0
0.75 1.0
m_h_3_000078 0.75 1.0
0.75 1.0
m_h_3_000079 0.75 1.0
0.75 1.0
m_h_3_000080 0.75 1.0
0.666666666667 0.666666666667
m_h_3_000081 0.666666666667 0.666666666667
0.75 1.0
m_h_3_000082 0.75 1.0
0.5 0.666666666667
m_h_3_00008

0.666666666667 0.666666666667
m_h_3_000237 0.666666666667 0.666666666667
0.666666666667 0.666666666667
m_h_3_000238 0.666666666667 0.666666666667
0.25 0.333333333333
m_h_3_000239 0.25 0.333333333333
0.75 1.0
m_h_3_000240 0.75 1.0
0.6 1.0
m_h_3_000241 0.6 1.0
0.75 1.0
m_h_3_000242 0.75 1.0
1.0 0.666666666667
m_h_3_000243 1.0 0.666666666667
0.75 1.0
m_h_3_000244 0.75 1.0
0.666666666667 0.666666666667
m_h_3_000245 0.666666666667 0.666666666667
0.666666666667 0.666666666667
m_h_3_000246 0.666666666667 0.666666666667
0.75 1.0
m_h_3_000247 0.75 1.0
1.0 0.666666666667
m_h_3_000248 1.0 0.666666666667
0.666666666667 0.666666666667
m_h_3_000249 0.666666666667 0.666666666667
0.75 1.0
m_h_3_000250 0.75 1.0
0.666666666667 0.666666666667
m_h_3_000251 0.666666666667 0.666666666667
0.75 1.0
m_h_3_000252 0.75 1.0
0.75 1.0
m_h_3_000253 0.75 1.0
0.75 1.0
m_h_3_000254 0.75 1.0
0.75 1.0
m_h_3_000255 0.75 1.0
0.75 1.0
m_h_3_000256 0.75 1.0
0.666666666667 0.666666666667
m_h_3_000257 0.666666666667 0.66666666

0.75 1.0
m_h_3_000413 0.75 1.0
0.75 1.0
m_h_3_000414 0.75 1.0
0.666666666667 0.666666666667
m_h_3_000415 0.666666666667 0.666666666667
0.6 1.0
m_h_3_000416 0.6 1.0
0.666666666667 0.666666666667
m_h_3_000417 0.666666666667 0.666666666667
0.75 1.0
m_h_3_000418 0.75 1.0
1.0 0.666666666667
m_h_3_000419 1.0 0.666666666667
0.666666666667 0.666666666667
m_h_3_000420 0.666666666667 0.666666666667
0.5 0.666666666667
m_h_3_000421 0.5 0.666666666667
0.25 0.333333333333
m_h_3_000422 0.25 0.333333333333
0.666666666667 0.666666666667
m_h_3_000423 0.666666666667 0.666666666667
0.333333333333 0.333333333333
m_h_3_000424 0.333333333333 0.333333333333
0.75 1.0
m_h_3_000425 0.75 1.0
0.75 1.0
m_h_3_000426 0.75 1.0
0.75 1.0
m_h_3_000427 0.75 1.0
0.75 1.0
m_h_3_000428 0.75 1.0
0.75 1.0
m_h_3_000429 0.75 1.0
1.0 0.666666666667
m_h_3_000430 1.0 0.666666666667
0.666666666667 0.666666666667
m_h_3_000431 0.666666666667 0.666666666667
0.666666666667 0.666666666667
m_h_3_000432 0.666666666667 0.666666666667
0.6 1.

In [52]:
(boxes,probs,cats) = nn.classify('./data/train_0/mis/images/m_0_3_000049.png', net, NN_PROB_THRESH) 

In [53]:
labels = read_label('./data/train_0/mis/labels/m_0_3_000049.txt')
label_boxes = [ kitti_2_box_format(l) for l in labels ]
ap, recall = average_precision(label_boxes, boxes, IOU_THRESH)

In [54]:
ap

0.75

In [55]:
recall

1.0

In [38]:
boxes

[array([ 584.81933594,  220.34779358,  129.04016113,  105.63809204], dtype=float32),
 array([ 589.07788086,  173.05867004,   62.38708496,   49.52755737], dtype=float32),
 array([ 635.69897461,  169.13327026,   56.59399414,   41.10015869], dtype=float32)]

In [42]:
gb

[[610.0, 145.0, 654.0, 177.0],
 [556.0, 144.0, 612.0, 192.0],
 [518.0, 179.0, 646.0, 265.0]]

In [56]:
net.close()

AttributeError: 'tuple' object has no attribute 'close'

In [5]:
e = pickle.load( open( "error_table", "rb" ) )

In [6]:
type(e)

instance

In [13]:
e.entries.keys()

['foreground0_width',
 'color_sample',
 'road_path',
 'environment',
 'road_id',
 'foreground0car_color',
 'foreground0car_orientation',
 'foreground0_x',
 'foreground0car_category',
 'road_type',
 'foreground0car_id',
 'foreground0type',
 'sharpness_sample',
 'contrast_sample',
 'foreground0car_path',
 'foreground0_height',
 'background_color',
 'foreground0_y',
 'brightness_sample']

In [3]:
num_cars=1
sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
sample, norm_sample = get_sample(0, 'halton', types=types, domains=domains, dims=sample_dims)

IndexError: list index out of range